# Fishing Vessels Scripts
Within are two main functions:
1. Updating the `master` sheet with new data
2. Integrating data from `master` sheet to a provided sheet.
With both you'll need to upload the files to Colab and update variable names in the code to match.

See comments in the code for any variables you may need to change with regards to filenames, script configuration, column names etc.

Both scripts output a new file after completion with name : `{ORIGINAL_FILE_NAME}_updated.csv`

## Update `master` with new data

In [ ]:
import pandas as pd

MASTER_FILE_NAME = "master.csv"
INPUT_DATA_FILE_NAME = "testing.csv"

df_master = pd.read_csv(MASTER_FILE_NAME, low_memory=False)
df_new_data = pd.read_csv(INPUT_DATA_FILE_NAME, low_memory=False)

# Trim any extra rows off input
df_new_data = df_new_data.dropna(how='all')

# Can skip ahead if you've already processed earlier indices
START_IDX = 0

# Use to test things are working before full sending
# -1 means go full dataset, any other # stops at that index
MAX_IDX = -1

# Set to True if you want to overwrite any existing values that may be there
DO_OVERWRITES = False

# Left side is `master` column name
# Right side is `input` sheet
COPY_OVER_DATA_COLUMN_MAPPING = {
    "Flag": "Flag",
    "MMSI": "id number"
    #...
}

# Prints every change being made
DEBUG = False

# Change these if your *inputting* sheet has different column names
VESSEL_NAME_KEY="Vessel Name"
IMO_KEY="IMO Number"
CALLSIGN_KEY="Callsign"

processed_rows = []
for idx, row_new in df_new_data.iterrows():
    if idx < START_IDX:
        continue

    name = row_new[VESSEL_NAME_KEY]
    imo = row_new[IMO_KEY]
    callsign = row_new[CALLSIGN_KEY]

    # Matches if all name, imo, and callsign are identical (ignoring capitalization)
    matching_rows_master = df_master.index[(df_master['Vessel Name'].str.lower() == name.lower()) \
                          & (df_master['IMO Number'] == str(int(imo))) \
                          & (df_master['Callsign'].str.lower() == callsign.lower())].tolist()

    if len(matching_rows_master) == 0 or all(row in processed_rows for row in matching_rows_master):
        not_found = len(matching_rows_master) == 0
        print(f"Skipping name: {name} imo: {imo} callsign: {callsign}, {'not found' if not_found else 'already processed'}")
        continue
    else :
        for row_number_master in matching_rows_master:
          """
            Do copy over logic here.

            e.g
          """
          for master_col, input_col in COPY_OVER_DATA_COLUMN_MAPPING.items():
            if pd.notna(row_new[input_col]) \
              and (DO_OVERWRITES or pd.isna(df_master.at[row_number_master, master_col])):

              df_master.at[row_number_master, master_col] = row_new[input_col]
              if (DEBUG) : print(f"Updated '{name}' at '{master_col}' with new value '{row_new[input_col]}'")

          processed_rows.append(row_number_master)

    if MAX_IDX != -1 and idx >= MAX_IDX:
        print ("Hit max index, writing and exiting...")
        break

output_name = f"{MASTER_FILE_NAME.replace('.csv', '')}_updated"
df_master.to_csv(f"{output_name}.csv", index=False)
print(f"Completed processing. Wrote to sheet: {output_name}.csv")


## Update inputted sheet with data **from** `master`
The script automatically attempts to copy over data *with matching column names*. This means only column names found in `master` will be transferred to the inputted sheet. In practice, this means if you want **all** available data from `master` to be transferred, you should add all non-matching column names to your input sheet.

e.g:

> I want my `peru-data.csv` to be updated with data from `master`. Specifically, I'm interested in augmenting my peru data with any SPRFMO links that `master` may have.

> To ensure this is copied over I add the following column to my `peru-data.csv`, as copied from `master`: `Vessel Authorization Link for the South Pacific Regional Fisheries Management Organisation`.

> Running the script then copies over any values found here in the case of a vessel match.

In [6]:
import pandas as pd

MASTER_FILE_NAME = "master.csv"
INPUT_DATA_FILE_NAME = "peru-data.csv"

master_df = pd.read_csv(MASTER_FILE_NAME, low_memory=False)
country_df = pd.read_csv(INPUT_DATA_FILE_NAME).astype(object)

# Trim any extra rows off input
country_df = country_df.dropna(how='all')

# Can skip ahead if you've already processed earlier indices
START_IDX = 0

# Use to test things are working before full sending
# -1 means go full dataset, any other # stops at that index
MAX_IDX = -1

# Set to True if you want to overwrite any existing values that may be there
DO_OVERWRITES = False

# Prints every change being made
DEBUG = False

# Change these if your *inputting* sheet has different column names
VESSEL_NAME_KEY="Vessel Name"
IMO_KEY="IMO Number"
CALLSIGN_KEY="Callsign"

processed_rows = []
total_changes = 0
# Iterate through rows in country
for idx, row in country_df.iterrows():
    if idx < START_IDX:
        continue

    name = row[VESSEL_NAME_KEY]
    imo = ''
    # If imo is a number, convert to an int (often from a float) to remove any trailing zero e.g '32343.0'
    if pd.notna(row[IMO_KEY]) and isinstance(row[IMO_KEY], (int, float)):
        imo = int(row[IMO_KEY])
    imo = str(imo)

    callsign = row[CALLSIGN_KEY]

    match_condition = (
        (master_df['Vessel Name'] == name) |
        (master_df['IMO Number'] == imo)
        # Add more `or` conditions here or use `&` to force all to match
    )
    if match_condition.any():
        if DEBUG: print(f"[MATCH] Found match for name: '{name}' imo: '{imo}'")
        for col in country_df.columns:
            # Update only empty fields in `sheet_df`
            if (pd.isna(row[col]) or DO_OVERWRITES) and col in master_df.columns:
                matching_row = master_df.loc[match_condition].iloc[0]  # First match in `master`
                if pd.notna(matching_row[col]):
                    country_df.at[idx, col] = matching_row[col]
                    total_changes += 1
                    if (DEBUG) : print(f"Updated '{name}' at '{col}' with value '{matching_row[col]}'")

# Save new country csv
output_name = f"{INPUT_DATA_FILE_NAME.replace('.csv', '')}_updated"
country_df.to_csv(f"{output_name}.csv", index=False)
print(f"Completed processing. Made {total_changes} total changes. Wrote to sheet: {output_name}.csv")


Completed processing. Made 980 total changes. Wrote to sheet: peru-data_updated.csv
